In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas_ml_quant
from test.config import DF_TEST

print(pandas_ml_quant.__version__)


with DF_TEST.model() as m:
    from sklearn.neural_network import MLPClassifier

    from pandas_ml_common import random_splitter
    from pandas_ml_common.sampling.cross_validation import KEquallyWeightEvents
    from pandas_ml_utils import SkModel, PostProcessedFeaturesAndLabels, ClassificationSummary

    fit = m.fit(
        SkModel(
            MLPClassifier(activation='tanh', hidden_layer_sizes=(60, 50), random_state=42, warm_start=True, max_iter=2),
            PostProcessedFeaturesAndLabels(
                features=
                    [
                        lambda df: df["Close"].ta.macd()._[['macd.*', 'signal.*']],
                        lambda df: df.ta.adx()._[['+DI', '-DM', '+DM']],
                        lambda df: df["Close"].ta.mom(),
                        lambda df: df["Close"].ta.apo(),
                        lambda df: df.ta.atr(),
                        lambda df: df["Close"].ta.trix(),
                    ],
                feature_post_processor=lambda df: df.ta.rnn(280),
                labels=[
                    lambda df: df["Close"].ta.future_bband_quantile(include_mean=False).ta.one_hot_encode_discrete()
                ],
                targets=[
                    lambda df: df["Close"].ta.bbands()[["lower", "upper"]]
                ]
            ),
            summary_provider=ClassificationSummary,
        ),
        random_splitter(test_size=0.4, seed=42),
        cross_validation=KEquallyWeightEvents(n_splits=3)
    )

fit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.2.0


/home/kic/sources/private/projects/pandas-quant/.venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
there is no cross validation processing implemented, all folds will naively be loop as if it ispart of the data set. Make sure the `warm_start` is set to true in your scikit model!


In [5]:
fit.model._history

-1             0             1             2    
    0   1         0   1         0   1         0   1
0 NaN NaN  1.961652 NaN  1.961652 NaN  1.961652 NaN
1 NaN NaN  1.882598 NaN  1.882598 NaN  1.882598 NaN
2 NaN NaN       NaN NaN  1.867791 NaN  1.867791 NaN
3 NaN NaN       NaN NaN       NaN NaN  1.850236 NaN